# Data Wrangling in Pandas
<img src='images/img_one.jpg' align=right>
The Journal of Data Science defines "data science" as almost everything that has something to do with data. In a job, this translates to using data to have an impact on the organization by adding value. Most commonly it is to use and apply the data to solve complex business problems. One of the most common steps taken in data science work is data wrangling. The following is a concise guide on how to go about exploring, manipulating and reshaping data in python using the <a href="https://pandas.pydata.org/pandas-docs/stable/">pandas</a> library.
 
We will explore a breast cancer data set (credits: <a href="https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original)">UCI</a>) and use pandas to clean, reshape, massage and give us a clean data set, all of this will help dramatically increase the quality of our data. Note: Data quality is KEY for optimal performance with machine learning algorithms.

The following pandas functionalities will be covered:
1. Data exploration — columns, unique values in a column, describe, duplicates
2. Dealing with missing values — quantifying missing values per column, filling & dropping missing values
3. Reshaping data — one hot encoding, pivot tables, joins, grouping and aggregating
4. Filtering data
5. Other — Making descriptive columns, element-wise conditional operations



## Data Exploration
Let us begin by reading in our dataset (csv file) into pandas and displaying the column names along with their data types. Also take a moment to view the entire dataset.

In [ ]:
import pandas as pd
pd.options.display.max_columns =None
pd.options.display.max_rows =40
filename = 'data/breast_cancer_data.csv'
df = pd.read_csv(filename)

In [ ]:
#start of by actually looking at your data set
df

In [ ]:
# What is the size of our dataset?
df.shape

In [ ]:
# Over here we see the columns names and their data types
df.dtypes

In the data we have the following columns as described by the source — Patient ID: id number, Clump Thickness: 1–10, Uniformity of Cell Size: 1–10, Uniformity of Cell Shape: 1–10, Marginal Adhesion: 1–10, Single Epithelial Cell Size: 1–10, Bare Nuclei: 1–10, Bland Chromatin: 1–10, Normal Nucleoli: 1–10, Mitoses: 1–10, Class: malignant or benign, Doctor name: 4 different doctors.

Based on this, we can assume that patient_id is a unique identifier, class is going to tell us whether the tumor is malignant (cancerous) or benign (not cancerous). The remaining columns are numeric medical descriptions of the tumor, except for the doctor_name which is a categorical feature.

Things to keep in mind — If our goal is to predict wether a tumor is cancerous or not based on the remaining features, we will have to one hot encode the categorical data and clean up the numerical data.
From our first output we see that bare_nuclei was read as an object data type although the description is numeric. Therefore we will need to change this.

To verify that our data matches up with the source we can use the describe option in pandas:

In [ ]:
# This provides some statistics on the numerical data
df.describe()

This neatly summarizes some statistical data for all numerical columns. For categorical data we can hand this by grouping together values:

In [ ]:
# This aggreates the data by its column names, then we pass the aggregation function (size = count)
df.groupby(by =['class', 'doctor_name']).size()

## Dealing with missing values

With every dataset it is vital to evaluate the missing values. How many are there? Is it an error? Are there too many missing values? Does a missing value have a meaning relative to its context?
We can sum up the total missing values using the following:

In [ ]:
#Dealing with missing values? How many np.nan per column?

df.isna().sum() 

Now that we have identified our missing values, we have a few options. We can fill them in with a certain value (zero, mean/max/median by column, string) or drop them by row. Since there are few missing values, we can drop the rows to avoid skewing the data in further analysis.

In [ ]:
# # fill with zero
# df = df.fillna(0) 

In [ ]:
df = df.dropna(axis = 0, how = 'any')  #drop rows with any column having np.nan values

#Rename columns
#df.rename(index =str, columns = {'patient_id':'patient_id'})
df

This allows us to drop rows with any missing values in them.

## Inspecting duplicates
To view repeating rows we can start off by looking at the number of unique values in each column.

In [ ]:
# Here we list all columns
df.columns

In [ ]:
len(df)

In [ ]:
# Its good to inspect your unique key identifier
df.nunique()

We see here that although there are 690 rows, there are only 637 unique patient_id’s. This could mean that some patient appear more than once in the dataset. To isolate these patients and view their data, we use the following:

In [ ]:
# This shows rows that show up more than once and have the exact same column values. 
df[df.duplicated(keep = 'last')]

In [ ]:
# # This shows all instances where patient_id shows up more than once, but may have varying column values
df[df.duplicated(subset = 'patient_id', keep =False)].sort_values('patient_id')

In [ ]:
#Now that I have seen that there are some duplicates, I am going to go ahead and remove any duplicate rows

#df = df.drop_duplicates(subset = None, keep ='first')

In [ ]:
repeat_patients = df.groupby(by = 'patient_id').size().sort_values(ascending =False)
repeat_patients

This shows that one patient shows up in the data 6 times!

## Filtering data
If we want to remove patients that show up more that 2 times in the data set.

In [ ]:
filtered_patients = repeat_patients[repeat_patients > 2].to_frame().reset_index()
filtered_df = df[~df.patient_id.isin(filtered_patients.patient_id)]
filtered_df

If we did not have the tilde (“~”) we would get all individuals that repeat more than twice. By adding a tilde the pandas boolean series is reversed and thus the resulting data frame is of those that do NOT repeat more than twice.

## Reshaping data
The dataset has elements of categorical data in the “doctor_name” column. To feed this data into a machine learning pipeline, we will need to convert it into a one hot encoded column. This can be done with a sci-kit learn package, however we will do it in pandas to demonstrate the pivoting and merging functionality. Start off by creating a new dataframe with the categorical data.

In [ ]:
categorical_df = df[['patient_id','doctor_name']]
categorical_df

In [ ]:
# This specifies all rows (':') and column name 'doctor_count'
categorical_df.loc[:,'doctor_count'] = 1

In [ ]:
categorical_df

We add a column an extra column to identify which doctor a patient deals with. Pivot this table so that we only have numerical values in the cells and the columns become the doctors’ name. Then fill in the empty cells with 0.

In [ ]:
doctors_one_hot_encoded = pd.pivot_table( categorical_df,
                                  index = categorical_df.index, 
                                  columns = ['doctor_name'], 
                                   values = ['doctor_count'] )
doctors_one_hot_encoded = doctors_one_hot_encoded.fillna(0)
doctors_one_hot_encoded

Then drop the multiIndex columns:


In [ ]:
doctors_one_hot_encoded.columns = doctors_one_hot_encoded.columns.droplevel()
doctors_one_hot_encoded

We can now join this back to our main table. Typically a left join in pandas looks like this:

`leftJoin_df = pd.merge(df1, df2, on ='col_name', how='left')`

However we are joining on the index so we pass the “left_index” and “right_index” option to specify that the join key is the index of both tables



In [ ]:
combined_df = pd.merge(df, doctors_one_hot_encoded, left_index = True,right_index =True, how ='left')
combined_df

We can drop the column that we no longer need by the following:

In [ ]:
combined_df = combined_df.drop(columns=['doctor_name'])
combined_df

## Row-wise Operations
Another key component in data wrangling is having the ability to conduct row-wise or column wise operations. Examples of this are; rename elements within a column based on its value and create a new column that yields a specific value based on multiple attributes within the row.

For this example lets create a new column that categorizes a patients cell as normal or abnormal based on its attributes. We first define our function and the operation that it will be doing.

In [ ]:
def celltypelabel(x):
    if ((x['cell_size_uniformity'] < 5) &      (x['cell_shape_uniformity'] < 5)):
        
        return('normal')
    else:
        return('abnormal')

Then we use the pandas apply function to run the celltypelabel(x) function on the dataframe.

In [ ]:
combined_df['cell_type_label'] = combined_df.apply(lambda x: celltypelabel(x), axis=1)

In [ ]:
combined_df

## Conclusion
Although some of these data manipulation steps can be done in SAS and excel. Doing it in python not only allows you to connect the data to vast open source resources in computer vision, machine and deep learning, but also for ETL automation purposes and more.

# Homework
1. Hot encode a new column in this dataset for cancerous (1) or not cancerous (0).